# Data Visualization with Modern Data Science

> Querying Data with SQL

Yao-Jen Kuo <yaojenkuo@ntu.edu.tw> from [DATAINPOINT](https://www.datainpoint.com)

In [1]:
%LOAD sqlite3 db=data/taiwan_election_2024.db timeout=2 shared_cache=true

## The Elements of a SQL Statement

## (Recap) What is a SQL statement

- A SQL statement is a combination of keywords, object names(e.g. databases/tables/columns/functions), constants, and operators.
- SQL keywords are NOT case sensitive.
- Semicolon `;` is the standard way to separate each SQL statement in database systems that allow more than one SQL statement to be executed in the same call.
- Indentations or new lines are OPTIONAL in SQL statements.

## A SQL statement

Trying to identify keywords, object names, constants, and operators, respectively.

In [2]:
SELECT district_id,
       polling_place,
       ROUND((effective_votes + wasted_votes)*1.0 / (issued_votes + remained_votes), 2) AS voter_turnout
  FROM polling_places
 LIMIT 5;

district_id,polling_place,voter_turnout
15035,1,0.59
15036,2,0.6
15037,3,0.56
15038,4,0.53
15039,5,0.54


## The list of SQL keywords

- [SQLite Keywords](https://www.sqlite.org/lang_keywords.html)
- [Standard SQL Keywords](https://www.w3schools.com/sql/sql_ref_keywords.asp)

## (Recap)Download and install SQLiteStudio

- Download links: <https://github.com/pawelsalawa/sqlitestudio/releases>
- Download installers: `.exe` for Windows, `.dmg` for macOS.
- For macOS users: Override system security by System Preferences > Security & Privacy > Open Anyway.

## (Recap)Downloading an established database [taiwan_election_2024.db](https://taiwan-election-2024.s3.ap-northeast-1.amazonaws.com/taiwan_election_2024.db)

## The list of tables within connected database

In [3]:
SELECT name
  FROM sqlite_master
 WHERE name NOT LIKE 'sqlite%';

name
districts
villages
parties
election_types
candidates
polling_places
presidents
regional_legislators
aboriginal_legislators
party_legislators


## The list of columns of a specified table

In [4]:
SELECT name
  FROM PRAGMA_TABLE_INFO('districts');

name
id
county
town
polling_place
vote_tallied_at


In [5]:
SELECT name
  FROM PRAGMA_TABLE_INFO('polling_places');

name
id
district_id
polling_place
effective_votes
wasted_votes
issued_votes
remained_votes
election_type_id


## Calculated fields

## What is a calculated field

A calculated field is generated via the data from existed fields in connected databases.

## Several ways to generate a calculated field

- Functions.
- Constants.
- Operators.
- `CASE` statement.

## What is a function

In the context of programming, a function is a named sequence of statements that performs a desired operation.

## Using functions with SQL

```sql
SELECT FUNCTION_NAME(columns/constants, parameters);
```

## Functions in SQLite

- Scalar functions.
    - For type checking.
    - For numerics.
    - For texts.
- Date and time functions.
- Aggregate functions.

## Useful scalar functions for type checking

- `TYPEOF(X)`: The function returns a string that indicates the datatype.
- `PRAGMA_TABLE_INFO(table_name)`: The function returns one row for each column in the named table. 

## `TYPEOF(X)` function for a certain column

In [6]:
SELECT TYPEOF(id),
       TYPEOF(county),
       TYPEOF(vote_tallied_at)
  FROM districts
 LIMIT 1;

TYPEOF(id),TYPEOF(county),TYPEOF(vote_tallied_at)
integer,text,text


## `PRAGMA_TABLE_INFO(table_name)` for an entire table

In [7]:
SELECT *
  FROM PRAGMA_TABLE_INFO('districts');

cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,0,NULL,1
1,county,CHAR(3),0,NULL,0
2,town,VARCHAR(200),0,NULL,0
3,polling_place,INTEGER,0,NULL,0
4,vote_tallied_at,CHAR(19),0,NULL,0


## Using constants to examine datatype

- `TEXT`
- `INTEGER`
- `REAL`
- `NULL`

## Using `AS` for alias of calculated fields

In [8]:
SELECT 'I am a TEXT' AS text_constant,
       5566 AS integer_constant,
       3.14159 AS real_constant,
       NULL AS null_constant;

text_constant,integer_constant,real_constant,null_constant
I am a TEXT,5566,3.14159,NULL


In [9]:
SELECT TYPEOF('I am a TEXT') AS text_constant,
       TYPEOF(5566) AS integer_constant,
       TYPEOF(3.14159) AS real_constant,
       TYPEOF(NULL) AS null_constant;

text_constant,integer_constant,real_constant,null_constant
text,integer,real,null


## Using `CAST(X AS datatype)` to convert data type for query results

Before casting.

In [10]:
SELECT effective_votes,
       wasted_votes,
       issued_votes,
       remained_votes
  FROM polling_places
 LIMIT 1;

effective_votes,wasted_votes,issued_votes,remained_votes
616,1,617,423


## Using `CAST(X AS datatype)` to convert data type for query results

After casting.

In [11]:
SELECT CAST(effective_votes AS REAL),
       CAST(wasted_votes AS REAL),
       CAST(issued_votes AS REAL),
       CAST(remained_votes AS REAL)
  FROM polling_places
 LIMIT 1;

CAST(effective_votes AS REAL),CAST(wasted_votes AS REAL),CAST(issued_votes AS REAL),CAST(remained_votes AS REAL)
616.0,1.0,617.0,423.0


## Operators

- Numeric operators
    - `+`, `-`, `*`, `/`
- Text operators
    - `||`
- Relational operators
    - `=`, `!=`, `>`, `>=`, `<`, `<=`, `IN`, `BETWEEN`, `IS NULL`
- Logical operators
    - `AND`, `OR`, `NOT`

## Beware of using `/` dividing integers

In [12]:
SELECT 2/5,
       2*1.0/5,
       2/5*1.0,
       2/(5*1.0);

2/5,2*1.0/5,2/5*1.0,2/(5*1.0)
0,0.4,0.0,0.4


## Using `||` to concatenate texts

In [13]:
SELECT 'Tony' || ' ' || 'Stark' AS ironman;

ironman
Tony Stark


## Other scalar functions for numerics and texts

<https://www.sqlite.org/lang_corefunc.html>

## Common scalar functions for date and time functions

- `DATE(X)`
- `TIME(X)`
- `DATETIME(X)`
- `STRFTIME(format, X)`

Source: <https://www.sqlite.org/lang_datefunc.html>

```sql
SELECT DATE('now') AS date_of_now;

SELECT TIME('now', 'localtime') AS time_of_now;

SELECT DATETIME('now', 'localtime') AS datetime_of_now;
```

## Common format strings for date/time/datetime

- `%d` day of month: 00.
- `%H` hour: 00-24.
- `%j` day of year: 001-366.
- `%m` month: 01-12.
- `%M` minute: 00-59.
- `%S` seconds: 00-59.
- `%w` day of week 0-6 with Sunday==0.
- `%W` week of year: 00-53.
- `%Y` year: 0000-9999.

```sql
SELECT DATE('now') AS date_of_now,
       TIME('now', 'localtime') AS time_of_now, 
       STRFTIME('%d', DATE('now')) AS day_part,
       STRFTIME('%H', TIME('now', 'localtime')) AS hour_part,
       STRFTIME('%j', DATE('now')) AS year_day,
       STRFTIME('%m', DATE('now')) AS month_part,
       STRFTIME('%M', TIME('now', 'localtime')) AS minute_part,
       STRFTIME('%S', TIME('now', 'localtime')) AS second_part,
       STRFTIME('%w', DATE('now')) AS weekday,
       STRFTIME('%W', DATE('now')) AS nth_week,
       STRFTIME('%Y', DATE('now')) AS year_part;
```

## `CASE` statement is a conditional expression

We can add some "if this, then that..." logic to a SQL statement.

```sql
CASE WHEN condition_1 THEN result_1
     WHEN condition_2 THEN result_2
     ELSE result_else END AS calculated_field_alias
```

## `CASE` statements can be used to

- Turn numeric values into categories.
- Turn categories into new categories.

## Turn numeric values into categories

In [14]:
SELECT DISTINCT number,
       CASE WHEN number = 1 THEN '1號候選人'
            WHEN number = 2 THEN '2號候選人'
            ELSE '3號候選人' END AS number_text
  FROM presidents;

number,number_text
1,1號候選人
2,2號候選人
3,3號候選人


## Turn categories into new categories

In [15]:
SELECT DISTINCT county,
       CASE WHEN county IN ('臺北市', '桃園市', '新北市', '臺中市', '臺南市', '高雄市') THEN '六都'
            ELSE '非六都' END AS county_type
  FROM districts;

county,county_type
南投縣,非六都
嘉義市,非六都
嘉義縣,非六都
基隆市,非六都
宜蘭縣,非六都
屏東縣,非六都
彰化縣,非六都
新北市,六都
新竹市,非六都
新竹縣,非六都


## We can also roughly divide the SQL functions into 2 sub categories

1. Scalar functions.
2. Aggregate functions.

## The difference between these 2 categories

The major difference is whether if the rows of output equals to the rows of input.

## Rows of input `county` equals to `SUBSTR(county)`

`SUBSTR()` is a scalar function.

In [16]:
SELECT DISTINCT county,
       SUBSTR(county, 1, 2)
  FROM districts;

county,"SUBSTR(county, 1, 2)"
南投縣,南投
嘉義市,嘉義
嘉義縣,嘉義
基隆市,基隆
宜蘭縣,宜蘭
屏東縣,屏東
彰化縣,彰化
新北市,新北
新竹市,新竹
新竹縣,新竹


## Aggregate functions combine values from multiple rows and return a single result based on an operation on those values

In [17]:
SELECT SUM(votes) AS total_president_votes
  FROM presidents;

total_president_votes
13947506


## Common aggregate functions

- `AVG(X)`
- `COUNT(X)`: the count of the number of times that X is not NULL.
- `COUNT(*)`: the total number of rows in the group.
- `MAX(X)`
- `MIN(X)`
- `SUM(X)`

Source: <https://www.sqlite.org/lang_aggfunc.html>

## How to count the number of columns of a table

By querying metadata!

In [18]:
SELECT COUNT(*) AS number_of_columns_for_districts
  FROM PRAGMA_TABLE_INFO('districts');

number_of_columns_for_districts
5


## Aggregating Data with `GROUP BY`

## `GROUP BY` keyword

`GROUP BY` on its own, eliminates duplicate values from the results, similar to the combination of `DISTINCT` and `ORDER BY`(in some RDBMS).

In [19]:
SELECT DISTINCT town
  FROM districts
 --ORDER BY town
 LIMIT 10;

town
中寮鄉
仁愛鄉
信義鄉
南投市
名間鄉
國姓鄉
埔里鎮
水里鄉
竹山鎮
草屯鎮


## `GROUP BY` keyword(Cont'd)

In [20]:
SELECT town
  FROM districts
 GROUP BY town
 LIMIT 10;

town
七堵區
七美鄉
七股區
三地門鄉
三峽區
三星鄉
三民區
三灣鄉
三義鄉
三芝區


## Using `GROUP BY` with `COUNT(*)`

In [21]:
SELECT county,
       COUNT(*) AS number_of_rows
  FROM districts
 GROUP BY county
 ORDER BY number_of_rows DESC;

county,number_of_rows
新北市,2681
高雄市,2038
臺中市,1902
臺北市,1764
臺南市,1548
桃園市,1384
彰化縣,1085
屏東縣,723
雲林縣,620
嘉義縣,535


## Using `GROUP BY` with `SUM()`

In [22]:
SELECT number,
       candidate_id,
       SUM(votes) AS sum_votes
  FROM presidents
 GROUP BY number;

number,candidate_id,sum_votes
1,330,3690466
2,331,5586019
3,329,4671021


## Filtering an aggregate query using `HAVING`

We are already familiar with using `WHERE` for filtering, but aggregate functions cannot be used in a `WHERE` statement because they operate at the row level, and aggregate functions work across rows.

## Combining `GROUP BY`, `COUNT(*)`, and `HAVING`

In [23]:
SELECT county,
       COUNT(*) AS number_of_rows
  FROM districts
 GROUP BY county
HAVING number_of_rows >= 1000
 ORDER BY number_of_rows DESC;

county,number_of_rows
新北市,2681
高雄市,2038
臺中市,1902
臺北市,1764
臺南市,1548
桃園市,1384
彰化縣,1085


## Combining `GROUP BY`, `SUM()`, and `HAVING`

In [24]:
SELECT number,
       candidate_id,
       SUM(votes) AS sum_votes
  FROM presidents
 GROUP BY number
HAVING sum_votes > 4500000;

number,candidate_id,sum_votes
2,331,5586019
3,329,4671021


## Sub-queries

## What is a sub-query?

A sub-query is nested inside another query, we just enclose the sub-query in parentheses and use it where needed.

## Useful sub-query skills

- Filtering with sub-queries in a `WHERE` statement.
- Creating a calculated field with sub-queries in a `SELECT` statement.
- Sub-queries in a `FROM` statement.

## Filtering with sub-queries in a `WHERE` statement

Which polling place has the most wasted votes in president election?

## Which polling place has the most wasted votes in president election?

- First query for the most wasted votes.
- Second query for the polling place.

In [25]:
SELECT MAX(wasted_votes) AS max_wasted_votes
  FROM polling_places
 WHERE election_type_id = 1;

max_wasted_votes
29


In [26]:
SELECT *
  FROM polling_places
 WHERE wasted_votes = 29 AND
       election_type_id = 1;

id,district_id,polling_place,effective_votes,wasted_votes,issued_votes,remained_votes,election_type_id
1401,13020,668,774,29,803,309,1


## Which polling place has the most wasted votes in president election?(Cont'd)

Sub-query: combining 2 queries into one.

In [27]:
SELECT *
  FROM polling_places
 WHERE wasted_votes = (SELECT MAX(wasted_votes) AS max_wasted_votes
                         FROM polling_places
                        WHERE election_type_id = 1) AND
       election_type_id = 1;

id,district_id,polling_place,effective_votes,wasted_votes,issued_votes,remained_votes,election_type_id
1401,13020,668,774,29,803,309,1


## Creating a calculated field with sub-queries in a `SELECT` statement

Calculating vote percentage in president election.

## Calculating vote percentage in president election

- First query for the number of total votes(denominator).
- Second query for the number of votes by candidates(numerator).

In [28]:
SELECT SUM(votes) AS number_of_total_votes
  FROM presidents;

number_of_total_votes
13947506


In [29]:
SELECT number,
       candidate_id,
       ROUND(SUM(votes) / 13947506.0, 4) AS vote_percentage
  FROM presidents
 GROUP BY number;

number,candidate_id,vote_percentage
1,330,0.2646
2,331,0.4005
3,329,0.3349


In [30]:
SELECT number,
       candidate_id,
       ROUND(SUM(votes) / (SELECT SUM(votes)*1.0 AS number_of_total_votes FROM presidents), 4) AS vote_percentage
  FROM presidents
 GROUP BY number;

number,candidate_id,vote_percentage
1,330,0.2646
2,331,0.4005
3,329,0.3349


## Sub-queries in a `FROM` statement

Another way to combine `GROUP BY`, `SUM()`, and not using `HAVING`.

In [31]:
SELECT number,
       candidate_id,
       sum_votes
  FROM (SELECT number,
               candidate_id,
               SUM(votes) AS sum_votes
          FROM presidents
         GROUP BY number)
 WHERE sum_votes > 4500000;

number,candidate_id,sum_votes
2,331,5586019
3,329,4671021


## Putting what we have so far all together

SQL is about the order of keywords, so follow this convention:

```sql
SELECT column_names,
       CASE WHEN conditions THEN result
            ...
            ELSE result_else END AS alias
  FROM table_name
 WHERE conditions
 GROUP BY column_names
HAVING aggregated_conditions
 ORDER BY column_names
 LIMIT m OFFSET m;
```